In [1]:
from datetime import datetime
from db import PythonProblems
import re

### Creating and connecting to database

In [4]:
db = PythonProblems('python.sqlite', 'init_tables.sql')

In [6]:
with open('100+ Python challenging programming exercises.txt', 'r', encoding='latin1') as python_file:
    content = python_file.read()
    
content = content.split('#----------------------------------------#')
problem_list = []
hs_list = []

# Remove introduction
del content[0]
del content[0]

# Get problem main area
for problem in content:
    # Replace typos:
    problem = problem.replace('Hint:', 'Hints:')
    problem = re.sub('Hints\n', 'Hints:', problem)
    
    # Remove empty lines
    if problem.strip() == "":
        continue
        
    problem_list.append(problem.split("Hints:")[0])
    hs_list.append(problem.split("Hints:")[1])

question_list = []
question_error = []
difficulty = []
# Separate question from level
for problem in problem_list:
    # Get level
    level = re.search('level (\d+)', problem.lower())
    try:
        difficulty.append(int(level.group(1)))
    except AttributeError:
        difficulty.append(level)
    
    # Remove extra questions and level
    problem = re.sub('Level \d+', '', problem)
    problem = re.sub('level \d+', '', problem)
    problem = re.sub('Question \d+', '', problem)
    problem = re.sub('Question£º', '', problem)
    problem = re.sub('^\d.\d', '', problem.strip())
    
    try:
        question_list.append(problem.split("Question:")[1])
    except IndexError:
        question_list.append(problem)
        
hint_list = []
solution_list = []
# Separate solution from hint
for solution in hs_list:
    # Replace typos:
    solution = solution.replace('Solutions:', 'Solution:')
    solution = re.sub('Solution\n', 'Solution:', solution)
    
    hint_list.append(solution.split('Solution:')[0])
    solution_list.append(solution.split('Solution:')[1])

problems = []
solutions = []
link = ("https://github.com/zhiwehu/Python-programming-exercises/blob/"
       "master/100%2B%20Python%20challenging%20programming%20exercises.txt")
# Create DB structure
for idx in range(len(problem_list)):
    problems.append({"title": "", "difficulty": difficulty[idx], "link": link,
                     "content": question_list[idx].strip(), "retrieved_date": datetime.now(),
                     "hint": hint_list[idx].strip(), "crawler": "100+PythonProblems"})
    if not difficulty[idx]:
        del problems[idx]["difficulty"]
    
    solutions.append({"content": solution_list[idx].strip(), "link": link,
                      "retrieved_date": datetime.now(), "ignore": False,
                      "idx": idx})

In [7]:
db.insert_rows(problems, solutions)
tp, ts, rp, rs = db.populate()
db.close_connection()
print("Got %d problems inserted, %d solutions inserted, %d problems repeated and %d solutions repeated" % (
      tp, ts, rp, rs))

Got 97 problems inserted, 97 solutions inserted, 3 problems repeated and 0 solutions repeated
